In [1]:
import sys
import pandas as pd
import numpy as np
from common import commons
home = commons.home
logger = commons.logger
import os
from heapq import nsmallest

def cal_beta(beta_file,pos_file):
    betas = pd.read_csv(beta_file,sep='\s+',index_col=['TargetID'])
    mean_betas = pd.DataFrame(betas.mean(axis=1),columns=['beta'])
    mean_betas.index = betas.index
    pos = pd.read_csv(pos_file,sep='\s+',usecols=[0,2,3],index_col=0, header=None,skiprows=1,names=['id','chr','coordinate'])
    beta_pos = mean_betas.join(pos)
    return beta_pos

INFO    ; 2019-05-30 04:41:42,317; root           ; commons:<module>;39: Existing logger at commons


In [2]:
commons.__file__

'/home/ec2-user/xiaobo/git/EnsembleCpG/code/common/commons.py'

In [3]:
type_name = commons.type_name  ## amyloid, cerad, tangles
with_cell_type = commons.with_cell_type ## with or without
logger.info('AD {}_{} training sites selection'.format(type_name,with_cell_type))
beta_file = home+'data/AD_CpG/ROSMAP_arrayMethylation_imputed.tsv'
pos_file = home+'data/AD_CpG/ROSMAP_arrayMethylation_metaData.tsv'
logger.info('calculating mean beta values for each AD site')
logger.info('beta values for each sample are saved in {}'.format(beta_file))

INFO    ; 2019-05-30 04:47:27,070; root           ; <ipython-input-3-4e7c427ed127>:<module>;3: AD braak_with training sites selection
INFO    ; 2019-05-30 04:47:27,072; root           ; <ipython-input-3-4e7c427ed127>:<module>;6: calculating mean beta values for each AD site
INFO    ; 2019-05-30 04:47:27,073; root           ; <ipython-input-3-4e7c427ed127>:<module>;7: beta values for each sample are saved in /home/ec2-user/xiaobo/git/EnsembleCpG/data/AD_CpG/ROSMAP_arrayMethylation_imputed.tsv


In [4]:
all_sites_betas = cal_beta(beta_file,pos_file)
pos_pvalues ={'amyloid':0.00005,'cerad':0.00001,'ceradaf':0.00005,'tangles':0.0000005,'cogdec':0.00003,'gpath':0.00001,'braak':0.00005}
### 0.001 for amyloid, 0.0001 for cerad, 0.00001 for tangles,0.002 for cogdec, 0.0002 for gpath,0.0002 for braak
pos_pvalue = pos_pvalues[type_name] 
logger.info('positive training sites pvalue threshold is <= {}'.format(pos_pvalue))
neg_pvalue = 0.4
logger.info('negative training sites pvalue threshold is >= {}'.format(neg_pvalue))
sample_ratio_neg_to_pos = 10
logger.info('ratio of negative training samples to positive samples are '.format(sample_ratio_neg_to_pos))

INFO    ; 2019-05-30 04:57:13,028; root           ; <ipython-input-4-a01d56d2e9c1>:<module>;5: positive training sites pvalue threshold is <= 5e-05
INFO    ; 2019-05-30 04:57:13,030; root           ; <ipython-input-4-a01d56d2e9c1>:<module>;7: negative training sites pvalue threshold is >= 0.4
INFO    ; 2019-05-30 04:57:13,031; root           ; <ipython-input-4-a01d56d2e9c1>:<module>;9: ratio of negative training samples to positive samples are 


In [5]:
all_sites_betas

,beta,chr,coordinate
TargetID,,,
cg00000165,0.154822,1,91194674
cg00000363,0.131544,1,230560793
cg00000957,0.779174,1,5937253
cg00001349,0.893716,1,166958439
cg00001364,0.727746,1,214170376
cg00001446,0.843326,1,43831041
cg00001534,0.882839,1,51034865
cg00001583,0.073302,1,200011786
cg00001593,0.927707,1,170490434


In [12]:
all_sites_file = home+'data/AD_CpG/Rosmap_'+type_name+'_ewas_'+with_cell_type+'celltype.csv'
logger.info('pvalues for AD {} sites are in {}'.format(type_name,all_sites_file))
all_sites = pd.read_csv(all_sites_file,usecols=[1,2,3],header=None,skiprows=1,index_col=0,names=['id','beta_sign','pvalue'])


INFO    ; 2019-05-30 05:18:36,588; root           ; <ipython-input-12-fcb5610385ee>:<module>;2: pvalues for AD braak sites are in /home/ec2-user/xiaobo/git/EnsembleCpG/data/AD_CpG/Rosmap_braak_ewas_withcelltype.csv


In [14]:
all_sites = all_sites.join(all_sites_betas).dropna()
all_sites.reset_index(inplace=True)


In [16]:
all_sites = all_sites[['id','chr','coordinate','beta_sign','pvalue','beta']]
all_sites['chr'] = all_sites['chr'].astype('i8')
all_sites = all_sites.query('chr<23')
all_sites.sort_values(['pvalue'],inplace=True,ascending=True)


In [17]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta
119979,cg05731218,2,216769199.0,3.478464e+01,5.741779e-09,0.859388
39932,cg05066959,8,41519308.0,3.309830e+01,1.312081e-08,0.865159
97325,cg06811467,1,154299179.0,3.075157e+01,4.165130e-08,0.085489
41616,cg17693222,8,42033472.0,2.872661e+01,1.134165e-07,0.867586
60280,cg07883124,13,113634042.0,2.860952e+01,1.201982e-07,0.705831
40821,cg11823178,8,41519399.0,2.790152e+01,1.708237e-07,0.919399
38784,cg25594100,7,4786943.0,2.699822e+01,2.677289e-07,0.780079
157640,cg25446460,4,176486240.0,2.685184e+01,2.879790e-07,0.898800
103395,cg13390284,1,65531864.0,2.657996e+01,3.297668e-07,0.128000
142475,cg16098726,3,128778738.0,2.613481e+01,4.117633e-07,0.836880


In [18]:
positive_sites = all_sites.query('pvalue<=@pos_pvalue')
positive_sites['label'] = np.where(positive_sites['beta_sign']>0,1,-1)
negative_sites = all_sites.query('pvalue>@neg_pvalue')
negative_sites['label'] = 0
negatives_sort_by_beta = negative_sites.sort_values(['beta'])

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
negatives_sort_by_beta

,id,chr,coordinate,beta_sign,pvalue,beta,label
65682,cg11525479,15,88799523.0,6.745179e-01,0.411762,0.001608,0
72265,cg01145420,17,37356856.0,2.549646e-01,0.613760,0.001680,0
79086,cg20530339,18,21594203.0,4.720907e-01,0.492255,0.001828,0
78947,cg17289202,18,56530789.0,1.173346e-01,0.732047,0.001832,0
71094,cg09876101,16,699357.0,4.329462e-01,0.510764,0.001865,0
46512,cg13220109,10,126150491.0,2.086293e-04,0.988480,0.001967,0
2369,cg07187135,1,235292131.0,4.959816e-07,0.999438,0.001982,0
8805,cg26662579,1,109969051.0,5.168255e-03,0.942710,0.002035,0
40314,cg08143636,8,92082417.0,4.470752e-02,0.832605,0.002087,0
66210,cg17356280,15,56657632.0,3.493202e-01,0.554690,0.002112,0


In [21]:
select_negs_list = []
hyper_sites = negatives_sort_by_beta.query('beta_sign>=0')
hypo_sites = negatives_sort_by_beta.query('beta_sign<0')
logger.info('sampling 10 negative (control) training sites with closest beta values to each positive training sites')
for beta,beta_sign in positive_sites[['beta','beta_sign']].values:
    tmp_sites = hyper_sites if beta_sign >=0 else hypo_sites
    negs = tmp_sites.loc[nsmallest(10, tmp_sites.index.values, key=lambda i: abs(tmp_sites.loc[i,'beta']-beta)),:]
    select_negs_list.extend(negs.values)
select_negs = pd.DataFrame(select_negs_list,columns=['id','chr','coordinate','beta_sign','pvalue','beta','label']).drop_duplicates(['chr','coordinate'])

INFO    ; 2019-05-30 05:48:20,796; root           ; <ipython-input-21-94e0eed751e9>:<module>;4: sampling 10 negative (control) training sites with closest beta values to each positive training sites


In [23]:
win_path = home+'data/commons/wins.txt'
logger.info('calculating 200bp window ids of positive and negative training sites')
pos_sites_with_winid, neg_sites_with_winid = commons.merge_with_feature_windows(win_path,positive_sites,select_negs)
all_sites_with_winid = pos_sites_with_winid.append(neg_sites_with_winid,ignore_index=True)
all_sites_with_winid.drop_duplicates(['id'],inplace=True)
all_sites_with_winid.sort_values(['chr','coordinate'],inplace=True) 

INFO    ; 2019-05-30 06:03:16,358; root           ; <ipython-input-23-1a4cab15bddf>:<module>;2: calculating 200bp window ids of positive and negative training sites
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ec2-user/xiaobo/git/EnsembleCpG/code/common/commons.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].astype('i8')
/home/ec2-user/xiaobo/git/EnsembleCpG/code/common/commons.py:124: SettingWithCopyWarning: 
A value is trying to be set

In [25]:
all_sites_with_winid

,id,chr,coordinate,beta_sign,pvalue,beta,label,start,winid,end
191,cg22381068,1,906363.0,0.002513,0.960036,0.241956,0,906201,4532,906400
192,cg24730157,1,984428.0,0.704868,0.401441,0.919376,0,984401,4923,984600
193,cg08480609,1,986560.0,0.011082,0.916192,0.783207,0,986401,4933,986600
0,cg04972348,1,1100035.0,17.611301,0.000031,0.643412,1,1100001,5501,1100200
194,cg14144728,1,1101462.0,0.157120,0.691943,0.788976,0,1101401,5508,1101600
195,cg08733913,1,1190778.0,0.556278,0.456015,0.944233,0,1190601,5954,1190800
196,cg24555071,1,1358016.0,0.570040,0.450498,0.594937,0,1358001,6791,1358200
197,cg24470133,1,1483206.0,0.176690,0.674363,0.960068,0,1483201,7417,1483400
1,cg22373622,1,1489544.0,21.924244,0.000003,0.918745,1,1489401,7448,1489600
198,cg15198247,1,1974844.0,0.448279,0.503375,0.677556,0,1974801,9875,1975000


In [26]:
with pd.HDFStore(home+'data/AD_CpG/'+type_name+with_cell_type+'/all_sites_winid','r') as h5s:
    all_sites_with_winid1 = h5s['all_sites_winid'] 

In [27]:
all_sites_with_winid1

,id,chr,coordinate,beta_sign,pvalue,beta,label,start,winid,end
191,cg22381068,1,906363.0,0.002513,0.960036,0.241956,0,906201,4532,906400
192,cg24730157,1,984428.0,0.704868,0.401441,0.919376,0,984401,4923,984600
193,cg08480609,1,986560.0,0.011082,0.916192,0.783207,0,986401,4933,986600
0,cg04972348,1,1100035.0,17.611301,0.000031,0.643412,1,1100001,5501,1100200
194,cg14144728,1,1101462.0,0.157120,0.691943,0.788976,0,1101401,5508,1101600
195,cg08733913,1,1190778.0,0.556278,0.456015,0.944233,0,1190601,5954,1190800
196,cg24555071,1,1358016.0,0.570040,0.450498,0.594937,0,1358001,6791,1358200
197,cg24470133,1,1483206.0,0.176690,0.674363,0.960068,0,1483201,7417,1483400
1,cg22373622,1,1489544.0,21.924244,0.000003,0.918745,1,1489401,7448,1489600
198,cg15198247,1,1974844.0,0.448279,0.503375,0.677556,0,1974801,9875,1975000


In [28]:
####450K data
logger.info('exporting all 450k window id of AD {}'.format(type_name))
all_sites = pd.read_csv(all_sites_file,usecols=[1,2,3],header=None,skiprows=1,index_col=0,names=['id','beta_sign','pvalue'])
all_sites = all_sites.join(all_sites_betas).dropna()
all_sites.reset_index(inplace=True)
all_sites = all_sites[['id','chr','coordinate','beta_sign','pvalue','beta']]
all_sites['chr'] = all_sites['chr'].astype('i8')
all_sites = all_sites.query('chr<23')
all_sites.sort_values(['pvalue'],inplace=True,ascending=True)

all_450k_sites_with_winid, __ = commons.merge_with_feature_windows(win_path,all_sites)
all_450k_sites_with_winid.drop(['beta_sign'],axis=1,inplace=True)


INFO    ; 2019-05-30 06:15:08,165; root           ; <ipython-input-28-219cbca16a67>:<module>;2: exporting all 450k window id of AD braak
/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/pandas/core/generic.py:7626: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/ec2-user/xiaobo/git/EnsembleCpG/code/common/commons.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data['chr'] = data['chr'].astype('i8')
/home/ec2-user/xiaobo/git/EnsembleCpG/code/common/commons.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [29]:
all_450k_sites_with_winid

,id,chr,coordinate,pvalue,beta,start,winid,end
0,cg13869341,1,15865.0,0.211443,0.877442,15801,80,16000
1,cg15560884,1,710097.0,0.861138,0.652625,710001,3551,710200
2,cg01014490,1,714177.0,0.249150,0.008987,714001,3571,714200
3,cg17505339,1,720865.0,0.035545,0.859844,720801,3605,721000
4,cg16736630,1,779995.0,0.010859,0.758187,779801,3900,780000
5,cg03128332,1,805338.0,0.650030,0.142779,805201,4027,805400
6,cg18147296,1,812539.0,0.363269,0.677963,812401,4063,812600
7,cg05597748,1,812679.0,0.065832,0.758387,812601,4064,812800
8,cg10988531,1,813317.0,0.056472,0.789541,813201,4067,813400
9,cg13938959,1,834183.0,0.013042,0.864894,834001,4171,834200
